In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
#pip install huggingface_hub openpyxl wandb bitsandbytes datasets accelerate loralib torch transformers peft tokenizers

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [5]:
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: deepak-krishbala. Use `wandb login --relogin` to force relogin


True

In [ ]:
#pip install torchvision --upgrade

In [6]:
import warnings
warnings.filterwarnings("ignore")
import os
import json
import bitsandbytes as bnb
import logging
from sklearn.model_selection import train_test_split
from functools import partial
from typing import Any, Dict, List, Tuple, Union
import pandas as pd
import click
import torch
import torch.nn as nn
import numpy as np
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    PreTrainedTokenizer,
    Trainer,
    TrainingArguments,
    TextGenerationPipeline,
    set_seed
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
logging.getLogger("py4j.clientserver").setLevel(logging.WARNING)

In [46]:
DEFAULT_SEED = 42

def apply_chat_template(row):
    instruction = "Generate Terraform code based on the provided requirements."
    user_query = row['user_query']
    cloud_provider = row['Cloud_provider']

    prompt = f"""You are a Terraform consultant with expertise in AWS, Azure, and GCP. Your task is to develop a Terraform configuration based on the provided user query for the specified cloud provider.

    - *User Query:* {user_query}
    - *Cloud Provider:* {cloud_provider}

    This Terraform configuration must include modules for essential components and design the modules to be reusable and easy to maintain.
    Adhere to the industry best practices for security and efficiency throughout the configuration. The output Terraform config should be in a structured format that is easy to understand and implement."""
    
    chat = [
        {"role": "system", "content": instruction},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": row['Terraform_Code']}
    ]
    
    formatted_prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

    return formatted_prompt

class DataCollatorForCompletionOnlyLM(DataCollatorForLanguageModeling):
    def torch_call(self, examples: List[Union[List[int], Any, Dict[str, Any]]]) -> Dict[str, Any]:
        batch = super().torch_call(examples)
        RESPONSE_KEY_NL="<|im_start|>assistant"
        # The prompt ends with the response key plus a newline.  We encode this and then try to find it in the
        # sequence of tokens.  This should just be a single token.
        response_token_ids = self.tokenizer.encode(RESPONSE_KEY_NL)

        labels = batch["labels"].clone()

        for i in range(len(examples)):

            response_token_ids_start_idx = None
            for idx in np.where(batch["labels"][i] == response_token_ids[0])[0]:
                response_token_ids_start_idx = idx
                break

            if response_token_ids_start_idx is None:
                raise RuntimeError(
                    f'Could not find response key {response_token_ids} in token IDs {batch["labels"][i]}'
                )

            response_token_ids_end_idx = response_token_ids_start_idx + 1

            # Make pytorch loss function ignore all tokens up through the end of the response key
            labels[i, :response_token_ids_end_idx] = -100

        batch["labels"] = labels

        return batch
    
def preprocess_batch(batch: Dict[str, List], tokenizer: AutoTokenizer, max_length: int) -> dict:
    return tokenizer(
        batch["chat_format"],
        max_length=max_length,
        truncation=True,
    )

def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed=DEFAULT_SEED) -> Dataset:
    """Loads the training dataset and tokenizes it so it is ready for training.
    Args:
        tokenizer (AutoTokenizer): Tokenizer tied to the model.
        max_length (int): Maximum number of tokens to emit from tokenizer.
    Returns:
        Dataset: HuggingFace dataset
    """

    dataset = load_training_dataset()

    logger.info("Preprocessing dataset")
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=["Category", "Cloud_provider", "user_query", "Terraform_Code","chat_format"],
    )

    # Make sure we don't have any truncated records, as this would mean the end keyword is missing.
    logger.info("Processed dataset has %d rows", dataset.num_rows)
    dataset = dataset.filter(lambda rec: len(rec["input_ids"]) < max_length)
    logger.info("Processed dataset has %d rows after filtering for truncated records", dataset.num_rows)

    logger.info("Shuffling dataset")
    dataset = dataset.shuffle(seed=seed)

    logger.info("Done preprocessing")

    return 

In [28]:
df=pd.read_excel('Research_Analysis_Base_data_filtered_new.xlsx')
del df['Unnamed: 0']
df.head()

,Category,Cloud_provider,user_query,Terraform_Code
0,GoogleCloudPlatform,GCP,Please generate a Terraform configuration for ...,#\n# Copyright 2020 Google LLC\n#\n# Licensed ...
1,immutability-io,AWS,Please generate a Terraform configuration for ...,# DHCP Options\n\n## Set Terraform version con...
2,forseti-security,GCP,Please generate a Terraform configuration for ...,/**\n * Copyright 2019 Google LLC\n *\n * Lice...
3,cncf,AWS,Please generate a Terraform configuration for ...,"# resource ""azurerm_network_security_group"" ""c..."
4,terraform-google-modules,GCP,Please generate a Terraform configuration for ...,"resource ""google_compute_ssl_certificate"" ""def..."


In [29]:
print(df.iloc[0]['user_query'])
print(df.iloc[0]['Terraform_Code'])

Please generate a Terraform configuration for setting up a Google Cloud Platform (GCP) project with necessary services, a VPC network, subnets, firewall rules, a NAT router, compute instances, instance templates, health checks, an instance group manager, and a global load balancer.
#
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#<!--* freshness: { owner: 'ttaggart@google.com' reviewed: '2020-sep-01' } *-->

 
variable "user_account" {
   type        = string
   des

In [31]:
model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [32]:
df['chat_format'] = df.apply(apply_chat_template, axis=1)

In [33]:
print(df['chat_format'].iloc[0])

<|im_start|>system
Generate Terraform code based on the provided requirements.<|im_end|>
<|im_start|>user
You are a Terraform consultant with expertise in AWS, Azure, and GCP. Your task is to develop a Terraform configuration based on the provided user query for the specified cloud provider.

    - *User Query:* Please generate a Terraform configuration for setting up a Google Cloud Platform (GCP) project with necessary services, a VPC network, subnets, firewall rules, a NAT router, compute instances, instance templates, health checks, an instance group manager, and a global load balancer.
    - *Cloud Provider:* GCP

    This Terraform configuration must include modules for essential components and design the modules to be reusable and easy to maintain.
    Adhere to the industry best practices for security and efficiency throughout the configuration. The output Terraform config should be in a structured format that is easy to understand and implement.<|im_end|>
<|im_start|>assistant


In [34]:
INSTRUCTION_PATH = "instruction_combined.jsonl"

In [35]:
dataset=Dataset.from_pandas(df)

In [36]:
dataset[0]

{'Category': 'GoogleCloudPlatform',
 'Cloud_provider': 'GCP',
 'user_query': 'Please generate a Terraform configuration for setting up a Google Cloud Platform (GCP) project with necessary services, a VPC network, subnets, firewall rules, a NAT router, compute instances, instance templates, health checks, an instance group manager, and a global load balancer.',
 'Terraform_Code': '#\n# Copyright 2020 Google LLC\n#\n# Licensed under the Apache License, Version 2.0 (the "License");\n# you may not use this file except in compliance with the License.\n# You may obtain a copy of the License at\n#\n#     https://www.apache.org/licenses/LICENSE-2.0\n#\n# Unless required by applicable law or agreed to in writing, software\n# distributed under the License is distributed on an "AS IS" BASIS,\n# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n# See the License for the specific language governing permissions and\n# limitations under the License.\n\n#<!--* freshness: { owne

In [38]:
DEFAULT_INPUT_MODEL ="Qwen/Qwen2.5-Coder-7B-Instruct"
max_length = 1024

In [39]:
model = AutoModelForCausalLM.from_pretrained(
    DEFAULT_INPUT_MODEL, 
    load_in_8bit=True, 
    device_map='auto'
)

config.json:   0%|          | 0.00/663 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

INFO:accelerate.utils.modeling:We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

In [40]:
for param in model.parameters():
    param.requires_grad = False  # freeze the model - train adapters later
    if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
        param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)
#model.lm_head = CastOutputToFloat(model.lm_head)
model.lm_head = CastOutputToFloat(model.lm_head)

In [41]:
peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
    )
model = get_peft_model(model, peft_config)

In [43]:
print("Found %d rows", dataset.num_rows)

Found %d rows 461


In [44]:
df.head()

,Category,Cloud_provider,user_query,Terraform_Code,chat_format
0,GoogleCloudPlatform,GCP,Please generate a Terraform configuration for ...,#\n# Copyright 2020 Google LLC\n#\n# Licensed ...,<|im_start|>system\nGenerate Terraform code ba...
1,immutability-io,AWS,Please generate a Terraform configuration for ...,# DHCP Options\n\n## Set Terraform version con...,<|im_start|>system\nGenerate Terraform code ba...
2,forseti-security,GCP,Please generate a Terraform configuration for ...,/**\n * Copyright 2019 Google LLC\n *\n * Lice...,<|im_start|>system\nGenerate Terraform code ba...
3,cncf,AWS,Please generate a Terraform configuration for ...,"# resource ""azurerm_network_security_group"" ""c...",<|im_start|>system\nGenerate Terraform code ba...
4,terraform-google-modules,GCP,Please generate a Terraform configuration for ...,"resource ""google_compute_ssl_certificate"" ""def...",<|im_start|>system\nGenerate Terraform code ba...


In [47]:
_preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=["Category", "Cloud_provider", "user_query", "Terraform_Code","chat_format"],
    )

processed_dataset = dataset.filter(lambda rec: len(rec["input_ids"]) < max_length)
print("Processed dataset has %d rows after filtering for truncated records", dataset.num_rows)

print("Shuffling dataset")
processed_dataset = processed_dataset.shuffle(seed=42)
print("Done preprocessing")

Map:   0%|          | 0/461 [00:00<?, ? examples/s]

Filter:   0%|          | 0/461 [00:00<?, ? examples/s]

Processed dataset has %d rows after filtering for truncated records 461
Shuffling dataset
Done preprocessing


In [48]:
for i in range(2):
    print(f"Entry {i}: {processed_dataset[i]}")
    print(len(processed_dataset[i]['input_ids']))

Entry 0: {'input_ids': [151644, 8948, 198, 31115, 49426, 627, 2038, 3118, 389, 279, 3897, 8502, 13, 151645, 198, 151644, 872, 198, 2610, 525, 264, 49426, 627, 30660, 448, 18726, 304, 23245, 11, 34119, 11, 323, 479, 7123, 13, 4615, 3383, 374, 311, 2225, 264, 49426, 627, 6546, 3118, 389, 279, 3897, 1196, 3239, 369, 279, 5189, 9437, 9109, 382, 262, 481, 353, 1474, 11361, 53386, 5209, 6923, 264, 49426, 627, 6546, 369, 6243, 705, 458, 23245, 14817, 74096, 448, 8392, 323, 21969, 11, 323, 60317, 264, 14817, 59458, 5611, 448, 458, 2551, 11540, 5733, 624, 262, 481, 353, 16055, 22916, 53386, 23245, 271, 262, 1096, 49426, 627, 6546, 1969, 2924, 13454, 369, 7565, 6813, 323, 2884, 279, 13454, 311, 387, 61571, 323, 4135, 311, 10306, 624, 262, 2410, 6739, 311, 279, 4958, 1850, 12378, 369, 4763, 323, 15024, 6814, 279, 6546, 13, 576, 2550, 49426, 627, 2193, 1265, 387, 304, 264, 32930, 3561, 429, 374, 4135, 311, 3535, 323, 4211, 13, 151645, 198, 151644, 77091, 198, 9813, 31521, 20627, 10086, 9813, 8970,

In [49]:
max_length

1024

In [50]:
split_dataset = processed_dataset.train_test_split(test_size=61, seed=42)
data_collator = DataCollatorForCompletionOnlyLM(
        tokenizer=tokenizer, mlm=False, return_tensors="pt", pad_to_multiple_of=8
    )

In [51]:
# Monitering the LLM
wandb.login(key ="acf1e89491f273cf4b165c4305f745f8ab7d2b36")
run = wandb.init(project='Terraform gneration using Qwen2.5-Coder-7B-Instruct', job_type="training", anonymous="allow")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/ec2-user/.netrc


In [52]:
# #Hyperparamter
# training_args = TrainingArguments(
#     output_dir='./ouput_qwen/terraform_finetuning',
#     logging_dir='./logging_qwen/terraform_finetuning',            
#     num_train_epochs=2,
#     per_device_train_batch_size=3,  # Decreased batch size
#     per_device_eval_batch_size=3, 
#     logging_steps=50,
#     save_steps=100,
#     warmup_steps=200, 
#     seed=42,
#     learning_rate=2e-4,  
#     evaluation_strategy="steps",
#     eval_steps=500,
#     fp16=True, 
#     report_to="wandb" 
# )

# model.config.use_cache = False  

In [55]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./output_qwen/terraform_finetuning',
    logging_dir='./logging_qwen/terraform_finetuning',            
    num_train_epochs=3, 
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2, 
    gradient_accumulation_steps=4,
    logging_steps=10,
    save_steps=50,
    warmup_ratio=0.1,
    seed=42,
    learning_rate=1e-5,
    evaluation_strategy="steps",
    eval_steps=50,
    fp16=True, 
    weight_decay=0.01,
    lr_scheduler_type="linear",
    max_grad_norm=1.0,
    report_to="wandb",
    run_name="terraform_finetuning_run_3_epochs"
)

model.config.use_cache = False

In [56]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_dataset["train"],
    eval_dataset=split_dataset["test"],
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

Step,Training Loss,Validation Loss


TrainOutput(global_step=45, training_loss=1.2854144308302138, metrics={'train_runtime': 553.752, 'train_samples_per_second': 0.645, 'train_steps_per_second': 0.081, 'total_flos': 1.1729097809436672e+16, 'train_loss': 1.2854144308302138, 'epoch': 3.0})

In [57]:
output_dir = "./model/qwen"
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('./model/qwen/tokenizer_config.json',
 './model/qwen/special_tokens_map.json',
 './model/qwen/vocab.json',
 './model/qwen/merges.txt',
 './model/qwen/added_tokens.json',
 './model/qwen/tokenizer.json')

In [58]:
trainer.model.push_to_hub("deepakkrishbala/Qwen2.5-Coder-7B-Instruct_finetuned")
tokenizer.push_to_hub("deepakkrishbala/Qwen2.5-Coder-7B-Instruct_finetuned")
print("Model and tokenizer have been pushed to Hugging Face Hub.")

adapter_model.safetensors:   0%|          | 0.00/80.8M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Model and tokenizer have been pushed to Hugging Face Hub.
